In [130]:
import pandas as pd
import numpy as np
#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer


In [131]:
df = pd.read_csv('/Users/noahpovis/Desktop/Buildweek22/Build Week 2 /DX.csv')


In [132]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


In [133]:
train.shape

(16278, 18)

In [134]:
val.shape

(5426, 18)

In [135]:
test.shape

(5427, 18)

In [136]:
def clean(i):
    i = i.copy()
    i.drop_duplicates(subset=['Organization Name','Funding Type'], inplace = True)
    i.drop_duplicates(keep='first', inplace= True)
    i['Raised Series A'] = np.where(i['Organization Name'].duplicated(), 'Yes', 'No')
    i.drop_duplicates(subset='Organization Name', keep='last', inplace= True)
    i['Money Raised Currency (in USD)'] = pd.to_numeric(i['Money Raised Currency (in USD)'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i['Announced Date'] = pd.to_datetime(i['Announced Date'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i  = i[i['Funding Type'] != 'Series A'] 
    return i 











In [137]:
train = clean(train)
val = clean(val)
test = clean(test)


In [138]:
train['Raised Series A'].value_counts()

No     12314
Yes      187
Name: Raised Series A, dtype: int64

In [139]:
train.shape , val.shape, test.shape

((12501, 19), (4539, 19), (4532, 19))

In [140]:
train['Investor Names'].isnull().sum()

3577

In [151]:
def tokenize(text):
    
    tokens = re.sub('[ˆa-zA-Z0-9]', '', text)
    tokens = tokens.lower().split()

    return tokens 


In [152]:
train = tokenize(train)
val = tokenize(val)
test = tokenize(test)


TypeError: expected string or bytes-like object

In [143]:
train.sample(5)

,Transaction Name,Transaction Name URL,Organization Name,Organization Name URL,Funding Type,Money Raised,Money Raised Currency,Money Raised Currency (in USD),Announced Date,Total Funding Amount,Total Funding Amount Currency,Total Funding Amount Currency (in USD),Organization Description,Organization Industries,Organization Location,Lead Investors,Number of Investors,Investor Names,Raised Series A
16059,Seed Round - LockinFuel,https://www.crunchbase.com/funding_round/locki...,LockinFuel,https://www.crunchbase.com/organization/lockin...,Seed,38000,USD,38000.0,2018-06-15,38000,USD,38000.0,LockinFuel is a fuel app that uses advanced ar...,"Apps, Artificial Intelligence","Sydney, New South Wales, Australia, Oceania",NaN,NaN,NaN,No
7268,Seed Round - NALA,https://www.crunchbase.com/funding_round/nala-...,NALA,https://www.crunchbase.com/organization/nala-m...,Seed,150000,USD,150000.0,2019-03-18,200000,USD,200000.0,NALA is an application interface that makes pa...,"Banking, Financial Services","Dar Es Salaam, Dar es Salaam, Tanzania, Africa",Y Combinator,5,"9Yards Capital, DFS Lab, The MBA Fund, Venture...",No
7736,Seed Round - Neurocea,https://www.crunchbase.com/funding_round/neuro...,Neurocea,https://www.crunchbase.com/organization/neurocea,Seed,NaN,NaN,NaN,2019-02-21,NaN,NaN,NaN,Neurocea is a biopharmaceutical company dedica...,"Biopharma, Biotechnology","Los Altos, California, United States, North Am...",NaN,NaN,NaN,No
15735,Seed Round - Shoppo,https://www.crunchbase.com/funding_round/shopp...,Shoppo,https://www.crunchbase.com/organization/shoppo,Seed,6000000,USD,6000000.0,2018-07-05,6000000,USD,6000000.0,Shoppo is an AI-driven global retail platform.,"Artificial Intelligence, E-Commerce","Dover, Delaware, United States, North America",GGV Capital,1,GGV Capital,No
16935,Seed Round - Supertripper,https://www.crunchbase.com/funding_round/super...,Supertripper,https://www.crunchbase.com/organization/supert...,Seed,NaN,NaN,NaN,2018-05-24,1000000,EUR,1119344.0,Supertripper is a global business management s...,Travel,"Paris, Ile-de-France, France, Europe",50 Partners,1,50 Partners,No


In [144]:
#def nlp_clean(z):
   # z['Organization Description'] = z['Organization Description'].apply(lambda x: x.lower())
   # z['Organization Industries'] = z['Organization Industries'].str.lower()
   # z['Organization Location'].astype(str)
   # z['Organization Location'] = z['Organization Location'].str.lower()
    #z['Investor Names'] = z['Investor Names'].str.lower()
    #z['Lead Investors'] = z['Lead Investors'].str.lower()
    #return z 
